**Báo cáo cuối kỳ môn học: PYTHON CHO KHOA HỌC DỮ LIỆU**

**Lớp 23TTH, Khoa Toán - Tin học, Trường Đại học Khoa học Tự nhiên, ĐHQG-HCM**

**Đề tài thực hiện:**
$$
\text{\textbf{USING DEEP LEARNING TO CLASSIFY ANIMAL AND HUMAN IMAGES}}
$$

**Giảng viên hướng dẫn: ThS. Hà Văn Thảo**

**Danh sách thành viên nhóm:**

1. 23110114 - Nguyễn Thị Hồng Thắm \
2. 23110123 - Lê Huỳnh Yến Vy \
3. 23110132 - Trần Nhật Anh

## GIỚI THIỆU

Object detection là một trong những chủ đề "nóng" trong deep learing bởi tính ứng dụng cao trong thực tiễn và nguồn dữ liệu dồi dào, dễ chuẩn bị. Một trong những thuật toán object detection nổi tiếng nhất là **YOLO**.

YOLO là mô hình mạng neuron tích chập (CNN) được sử dụng phổ biển để nhận dạng các đối tượng trong ảnh hoặc video. Điểm đặc biệt của mô hình này là có khả năng phát hiện tất cả các đối tượng trong một hình ảnh chỉ qua một lần lan truyền của CNN.

Các phương pháp truyền thống tách biệt bước đề xuất vùng và bước phân loại, YOLO xử lý đầu vào, vừa phân loại được các đối tượng, vừa dự đoán được vị trí của chúng trong một lần duy nhất.

YOLO có nghĩa là "You only look once", nghĩa là chỉ cần "nhìn" một lần là thuật toán đã có thể phát hiện được vật thể, cho thấy độ nhanh của thuật toán gần như là real-time.

Ứng dụng của YOLO cũng như nhiều thuật toán object detection khác, rất đa dạng: quản lý giao thông, đếm số sản phẩm trên băng chuyền nhà máy, đếm số vật nuôi trong chăn nuôi, phát hiện vật thể nguy hiểm (súng, dao,...), chấm công tự động,...

## TẠO MÔI TRƯỜNG ẢO VÀ KERNEL CHẠY NOTEBOOK (LINUX)

Dự án Python cần **môi trường ảo (virtual environment)** để tự cách ly, tránh xung đột phiên bản thư viện giữa các dự án. `venv` là môi trường ảo mà chúng ta sẽ sử dụng trong dự án này. Sau khi cài đặt `venv`, chúng ta di chuyển đường dẫn đến folder chứa dự án trong terminal và sử dụng lệnh sau để cài đặt môi trường ảo cho dự án:

`python -m venv .venv`

Trong đó, `.venv` là tên của folder chứa môi trường ảo của dự án, đồng thời nó cũng sẽ "đóng băng" phiên bản Python, pip và các thư viện sẽ được dùng trong dự án.

Kích hoạt môi trường ảo:

`source .venv/bin/activate`

Lúc này, phiên bản Python và `pip` được dùng là của môi trường ảo, các thư viện cài bằng `pip install` cũng chỉ ảnh hưởng trong `.venv`. Cách nhận biết đang ở môi trường ảo là promt terminal thường đổi thành `(.venv) user_name@machine:~` (nếu đang sử dụng Linux). Khi đã kích hoạt môi trường ảo, đảm bảo phiên bản Python và `pip` đã "đóng băng" trong đó, sử dụng lệnh:

`which python && which pip`

Nếu output có dạng `.../<project_name>/.venv/...` thì môi trường ảo đã được kích hoạt thành công.

Tiếp theo, tạo một kernel để chạy Jupyter Notebook. Cài đặt `ipykernel` để tạo kernel:

`python -m pip install ipykernel`

Sau khi cài đặt thành công, tiến hành tạo kernel để chạy file `.ipynb`:

`python -m ipykernel install --prefix .venv --name yolovenv --display-name "this_project"`

`--prefix .venv`: kernel mặc định không tự lưu vào `.venv`, thuộc tính này sẽ lưu kernel đã tạo vào `.venv`  
`--name yolovenv`: tên folder chứa kernel, ở đây tên folder là `yolovenv`. Kernel sẽ được lưu tại `.venv/share/jupyter/kernels/yolovenv/`  
`--display-name "this_project`: kernel sẽ hiển thị dưới tên `this_project` trong VS Code.

Khi đã tạo kernel, click vào biểu tượng kernel ở góc trên bên phải, chọn
$$
\text{Select Another Kernel} \rightarrow \text{Jupyter Kernel...} \rightarrow \text{this\_project}
$$

# KHAI BÁO THƯ VIỆN

In [ ]:
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from ultralytics import YOLO

# CHUẨN BỊ DỮ LIỆU

Dữ liệu được tải về tại các nguồn sau: \
- https://www.kaggle.com/datasets/antoreepjana/animals-detection-images-dataset \
- https://www.kaggle.com/datasets/biancaferreira/african-wildlife \
- https://www.kaggle.com/datasets/wutheringwang/dog-face-detectionyolo-format \
- https://www.kaggle.com/datasets/samuelayman/cat-dataset\
- https://universe.roboflow.com/labo-yolo/age-and-gender-xlnfj/dataset/3

Dữ liệu sau khi được tải về sẽ được xử lý (gán lại class ID; phân loại thành các folder train, valid, test;...), sau đó được gộp thành một folder tập dữ liệu (dataset) duy nhất. Dataset nếu muốn được YOLO "hiểu" thì phải có file `.yaml` chứa các thông tin về dataset như vị trí của file train, valid, test; số class; tên của các class ứng với mỗi class_id;...

Dataset của dự án này nằm tại `dataset/complete_dataset`, bao gồm: \
- folder `images` chứa các file ảnh, được chia thành 3 folder train, valid, test \
- folder `labels` chứa các file **nhãn dữ liệu (label)** có đuôi `.txt`, cũng được chia thành 3 folder train, valid, test và các file label có tên ứng với các file ảnh
- file `.yaml` để cung cấp thông tin về dataset cho YOLO

Sau khi hoàn tất xử lý dataset, dùng hàm `count_missing` để kiểm tra xem với mỗi file ảnh thì có file label tương ứng hay không.

In [3]:
def count_missing(BASE_DIR, IMAGE_EXTS, sub_folder):
    images_dir = BASE_DIR / "images" / sub_folder
    labels_dir = BASE_DIR / "labels" / sub_folder

    images_cnt = 0
    missing_cnt = 0

    for image in images_dir.iterdir():
        if image.suffix.lower() not in IMAGE_EXTS:
            continue

        images_cnt += 1
        """
        image.stem dùng để lấy tên của file image mà không có phần mở rộng
        Ví dụ: example123.png -> example123
        """
        label_file = labels_dir / f"{image.stem}.txt"
        if not label_file.exists():
            missing_cnt += 1

    return images_cnt, missing_cnt

Kiểm tra số ảnh bị thiếu label tương ứng.

In [30]:
BASE_DIR = Path("dataset/complete_dataset")
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".webp"}
sub_folders = ["train", "valid", "test"]

print(f"Checking dataset: {str(BASE_DIR)}\n")

total_images = 0
total_missing = 0
images_cnt_arr = np.array([])
missing_cnt_arr = np.array([])
percent_arr = np.array([])

for sub_folder in sub_folders:
    images_cnt, missing_cnt = count_missing(BASE_DIR, IMAGE_EXTS, sub_folder)
    total_images += images_cnt
    total_missing += missing_cnt

    percent = (missing_cnt / images_cnt) * 100 if images_cnt > 0 else 0

    images_cnt_arr = np.append(images_cnt_arr, images_cnt)
    missing_cnt_arr = np.append(missing_cnt_arr, missing_cnt)
    percent_arr = np.append(percent_arr, percent)

np_table = np.array([images_cnt_arr, missing_cnt_arr, percent_arr])
table = pd.DataFrame(np_table).transpose()
table.columns = ["images", "missing_labels", "missing_labels (%)"]
table.index = ["train", "valid", "test"]
table['images'] = table['images'].astype(int)
table['missing_labels'] = table['missing_labels'].astype(int)

missing_percent = (total_missing / total_images) * 100 \
    if total_images > 0 else 0

print(table)
print("")
print(f"-> Total images:       {total_images}")
print(f"-> Total missing:      {total_missing}")
print(f"-> Total missing (%):  {missing_percent}")

Checking dataset: dataset/complete_dataset

       images  missing_labels  missing_labels (%)
train   20451               0                 0.0
valid    6078               0                 0.0
test     4637               0                 0.0

-> Total images:       31166
-> Total missing:      0
-> Total missing (%):  0.0


Theo kết quả được in ra, dataset có tổng cộng 31166 ảnh và mỗi ảnh đều có file label tương ứng. Như vậy, dataset đã đúng với format của YOLO.

# KHAI BÁO, HUẤN LUYỆN VÀ LƯU MÔ HÌNH

Hàm train model.

In [ ]:
def train_model(model, epochs, imgsz, dataset_path, project_path, project_name):
    model.train(
        exist_ok=True,
        data=dataset_path,
        project=project_path,
        name=project_name,

        epochs=epochs,
        imgsz=imgsz,
        batch=8,
        workers=4,
        device=0,  # Sử dụng GPU
    )


def train_model_with_optimization(model, epochs, imgsz, dataset_path, 
                                  project_path, project_name):
    model.train(
        exist_ok=True,
        data=dataset_path,
        project=project_path,
        name=project_name,

        epochs=epochs,
        imgsz=imgsz,
        batch=8,
        workers=4,
        device=0,  # Sử dụng GPU

        optimizer="SGD",
        lr0=0.005,  # Tốc độ học ban đầu của model
        momentum = 0.937,  # Quán tính học
        weight_decay = 0.0005,  # Chống overfitting, weight quá lớn

        mosaic = 0.25,  # Ghép 4 ảnh thành một
        mixup = 0.0,  # Không cho phép trộn 2 ảnh và label lên nhau
        copy_paste = 0.0  # Không cho phép cắt object dán sang ảnh khác
    )

Sử dụng model YOLO11s và chuẩn bị các tham số để chuẩn bị train.

In [ ]:
BASE_MODEL = "yolo11s.pt"

epochs_list = [30, 50, 80]
imgsz_list = [448, 512]
with_optimization_list = [0, 1]

# Train model với 12 tham số khác nhau
parameters = list(product(epochs_list, imgsz_list, with_optimization_list))

Train model với 12 tham số.

In [ ]:
for parameter in parameters:
    epochs, imgsz, with_optimization = parameter

    """
    Lưu model tại "runs/yolo11s_custom/epochs{...}_imgsz{...}_optimization{...}"
    sau khi train
    """
    model_path = (
        Path(f"runs/yolo11s_custom")
        / f"epochs{epochs}_imgsz{imgsz}_optimization{with_optimization}"
        / "weights"
        / "best.pt"
    )
    project_path = "runs/yolo11s_custom"
    project_name = f"epochs{epochs}_imgsz{imgsz}_optimization{with_optimization}"

    # Train mô hình với dataset
    if model_path.exists():
        print(project_name)
        print(f"Model has been trained already. It is being loaded again: \
              {model_path}")
        model = YOLO(str(model_path))
    else:
        print(project_name)
        print("Model hasn't been trained. Start training...")

        # Train model dựa trên model gốc là YOLO11s
        model = YOLO(BASE_MODEL)
        if with_optimization:
            train_model_with_optimization(
                model=model,
                epochs=epochs,
                imgsz=imgsz,
                dataset_path=DATASET_PATH,
                project_path=project_path,
                project_name=project_name
            )
        else:
            train_model(
                model=model,
                epochs=epochs,
                imgsz=imgsz,
                dataset_path=DATASET_PATH,
                project_path=project_path,
                project_name=project_name
            )

        # Load lại best.pt sau khi train, nếu không tìm thấy thì in ra lỗi
        assert model_path.exists(), "Training finished but file best.pt not found"
        model = YOLO(str(model_path))

        print("Training finished")

    # Kiểm tra và khoá model với model gốc là YOLO11s
    # assert model.model.yaml['name'] == 'yolo11s'
    # model.info()

## TEST